In [ ]:
import scirpy as ir

In [ ]:
import scanpy as sc

In [ ]:
from glob import glob
import warnings
import tarfile

import anndata
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc

In [ ]:
import muon as mu

In [ ]:
import numpy as np

## TCR

In [ ]:
samples = {
    "10mix1": {"group": "10mix"},
    "10mix2": {"group": "10mix"},
    "11mix1": {"group": "11mix"},
    "11mix2": {"group": "11mix"},
    "GF1": {"group": "GF"},
    "GF2": {"group": "GF"},

}    
    
# Create a list of AnnData objects (one for each sample)
adatas_tcr = {}
#adatas_gex = {}
for sample, sample_meta in samples.items():
    #adata_gex = sc.read_10x_h5(f"/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2019-10-29_sorted_cd8/analyses_icbi/{sample}/outs/per_sample_outs/{sample}/count/sample_filtered_feature_bc_matrix.h5")
    adata_tcr = ir.io.read_10x_vdj(f"/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2019-10-29_sorted_cd8/analyses_icbi/{sample}/outs/per_sample_outs/{sample}/vdj_t/filtered_contig_annotations.csv")
    # concatenation only works with unique gene names
    #adata_gex.var_names_make_unique()
    adatas_tcr[sample] = adata_tcr
    #adatas_gex[sample] = adata_gexc

In [ ]:
samples = {
    "10mix-ICI1": {"group": "10mix", "condition":"ICI"},
    "10mix-ICI2": {"group": "10mix","condition":"ICI"},
    "11mix-ICI1": {"group": "11mix","condition":"ICI"},
    "11mix-ICI2": {"group": "11mix","condition":"ICI"},
    "GF-ICI1": {"group": "GF","condition":"No ICI"},
    "GF-ICI2": {"group": "GF","condition":"No ICI"},
    "GF-ICI1-plus": {"group": "GF-plus","condition":"ICI"},
    "GF-ICI2-plus": {"group": "GF-plus","condition":"ICI"},
}    
    
# Create a list of AnnData objects (one for each sample)
adatas_tcr_2021 = {}
#adatas_gex_2021 = {}
for sample, sample_meta in samples.items():
    #adata_gex_2021 = sc.read_10x_h5(f"/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/{sample}/outs/per_sample_outs/{sample}/count/sample_filtered_feature_bc_matrix.h5")
    adata_tcr_2021 = ir.io.read_10x_vdj(f"/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/{sample}/outs/per_sample_outs/{sample}/vdj_t/filtered_contig_annotations.csv")
    # concatenation only works with unique gene names
    #adata_gex_2021.var_names_make_unique()
    adatas_tcr_2021[sample] = adata_tcr_2021
    #adatas_gex_2021[sample] = adata_gex_2021

In [ ]:
samples = {
    "10mix-ICI1": {"group": "10mix", "condition":"ICI"},
    "10mix-ICI2": {"group": "10mix","condition":"ICI"},
    "11mix-ICI1": {"group": "11mix","condition":"ICI"},
    "11mix-ICI2": {"group": "11mix","condition":"ICI"},
    "GF-ICI1": {"group": "GF","condition":"No ICI"},
    "GF-ICI2": {"group": "GF","condition":"No ICI"},
    "GF-ICI1-plus": {"group": "GF-plus","condition":"ICI"},
    "GF-ICI2-plus": {"group": "GF-plus","condition":"ICI"},
    "10mix1": {"group": "10mix", "condition":"naive"},
    "10mix2": {"group": "10mix", "condition":"naive"},
    "11mix1": {"group": "11mix", "condition":"naive"},
    "11mix2": {"group": "11mix", "condition":"naive"},
    "GF1": {"group": "GF", "condition":"naive"},
    "GF2": {"group": "GF", "condition":"naive"},
}    

In [ ]:
# Merge anndata objects
#adata_gex = anndata.concat(adatas_gex, index_unique="_")
adata_tcr = anndata.concat(adatas_tcr, index_unique="_")

In [ ]:
# Merge anndata objects
#adata_gex_2021 = anndata.concat(adatas_gex_2021, index_unique="_")
adata_tcr_2021 = anndata.concat(adatas_tcr_2021, index_unique="_")

In [ ]:
#adata_gex_merge = anndata.concat([adata_gex,adata_gex_2021], index_unique="_")
adata_tcr_merge = anndata.concat([adata_tcr,adata_tcr_2021], index_unique="_")

In [ ]:
# Set global met'adata on `mdata.obs`
adata_tcr_merge.obs["sample_id"] = adata_tcr_merge.obs_names.to_series().str.split("_", expand=True)[1]
adata_tcr_merge.obs["group"] = adata_tcr_merge.obs["sample_id"].map(lambda x: samples[x]["group"])
adata_tcr_merge.obs["condition"] = adata_tcr_merge.obs["sample_id"].map(lambda x: samples[x]["condition"])

In [ ]:
def remove_after_last_underscore(name):
    return name.rsplit('_', 1)[0]

# Apply the function to obs_names
adata_tcr_merge.obs_names = [remove_after_last_underscore(name) for name in adata_tcr_merge.obs_names]

In [ ]:
adata_tcr_merge.obs.group.unique()

In [ ]:
mask = ~(adata_tcr_merge.obs['sample_id'] == "GF2") #remove GF2 because the quality of this sample was not good
# Subset the AnnData object using the boolean mask
adata_tcr_merge = adata_tcr_merge[mask].copy()

In [ ]:
set(adata_tcr_merge.obs["sample_id"])

In [ ]:
adata_tcr_merge.obs_names

## GEX

In [ ]:
adata_gex =  sc.read_h5ad("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/results/adata_solo_integrated.h5ad")

In [ ]:
mask =  ~(adata_gex.obs['sample_id'] == "GF2") #remove GF2 because the quality of this sample was not good
# Subset the AnnData object using the boolean mask
adata_gex = adata_gex[mask].copy()

In [ ]:
adata_gex

In [ ]:
adata_gex.obs[["COND", "batch_id", "condition","sample_id"]]

In [ ]:
adata_gex.obs["group"] = adata_gex.obs["sample_id"].map(lambda x: samples[x]["group"])
adata_gex.obs["condition"] = adata_gex.obs["sample_id"].map(lambda x: samples[x]["condition"])

In [ ]:
new_obs_names = [f"{name}_{sample_id}" for name, sample_id in zip(adata_gex.obs_names, adata_gex.obs['sample_id'])]

# Update the obs_names with the new names
adata_gex.obs_names = new_obs_names

In [ ]:
def replace_after_hyphen(name):
    parts = name.split('-', 1)  # Split at the first hyphen
    if len(parts) > 1 and len(parts[1]) > 0:
        parts[1] = '1' + parts[1][1:]  # Replace the first character after the hyphen
    return '-'.join(parts)

# Apply the function to obs_names
adata_gex.obs_names = [replace_after_hyphen(name) for name in adata_gex.obs_names]

In [ ]:
adata_gex.var.index.name = None

## MUDATA

In [ ]:
mdata = mu.MuData({"gex": adata_gex, "airr": adata_tcr_merge})

In [ ]:
mdata

### MUDATA QC

In [ ]:
mu.pl.embedding(
    mdata,
    basis="gex:umap",
    color=["gex:leiden"],
    ncols=3,
    wspace=0.7,
)

In [ ]:
# mitochondrial genes
mdata["gex"].var["mt"] = mdata.var_names.str.startswith("mt-")


In [ ]:
sc.pp.calculate_qc_metrics(
    mdata["gex"], qc_vars=["mt"], inplace=True, percent_top=[20], log1p=True
)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
axes=axes.flatten()
sc.pl.umap(
    mdata["gex"],
    color="pct_counts_mt",
    cmap="inferno",
    vmin=0,
    vmax="p99",
    sort_order=False,
    show=False,
    ax=axes[0],
)
sc.pl.umap(
    mdata["gex"],
    color="total_counts",
    cmap="inferno",
    vmax="p99",
    sort_order=False,
    show=False,
    ax=axes[1],
)
sc.pl.umap(
    mdata["gex"],
    color="n_genes_by_counts",
    cmap="inferno",
    vmax="p99",
    sort_order=False,
    show=False,
    ax=axes[2],
)
sc.pl.umap(
    mdata["gex"],
    color="n_genes",
    cmap="inferno",
    vmax="p99",
    sort_order=False,
    show=False,
    ax=axes[3],
)

In [ ]:
def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (
        np.median(M) + nmads * median_abs_deviation(M) < M
    )
    return outlier

In [ ]:

from scipy.stats import median_abs_deviation

In [ ]:
mdata["gex"].obs["outlier"] = (
    is_outlier(mdata["gex"], "log1p_total_counts", 1)
    | is_outlier(mdata["gex"], "log1p_n_genes_by_counts", 1)
    | is_outlier(mdata["gex"], "pct_counts_in_top_20_genes",1)
)
mdata["gex"].obs.outlier.value_counts()

In [ ]:
mdata_qc = mdata["gex"][(~mdata["gex"].obs.outlier)]

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
axes=axes.flatten()
sc.pl.umap(
    mdata_qc,
    color="pct_counts_mt",
    cmap="inferno",
    vmin=0,
    vmax="p99",
    sort_order=False,
    show=False,
    ax=axes[0],
)
sc.pl.umap(
    mdata_qc,
    color="total_counts",
    cmap="inferno",
    vmax="p99",
    sort_order=False,
    show=False,
    ax=axes[1],
)
sc.pl.umap(
    mdata_qc,
    color="n_genes_by_counts",
    cmap="inferno",
    vmax="p99",
    sort_order=False,
    show=False,
    ax=axes[2],
)
sc.pl.umap(
    mdata_qc,
    color="n_genes",
    cmap="inferno",
    vmax="p99",
    sort_order=False,
    show=False,
    ax=axes[3],
)

In [ ]:
#mdata_qc.obs["cell_type_coarse"] = mdata_qc.obs.leiden.map(cl_annotation)

In [ ]:
ir.pp.index_chains(mdata)

In [ ]:
ir.tl.chain_qc(mdata)

In [ ]:
mdata_colon  = mdata[mdata.obs["gex:sample_id"].isin(['10mix1',
 '10mix2',
 '11mix1',
 '11mix2',
  'GF1'])]

In [ ]:
mdata_til  = mdata[mdata.obs["gex:sample_id"].isin(['10mix-ICI1',
 '10mix-ICI2',
 '11mix-ICI1',
 '11mix-ICI2',
 'GF-ICI1',
 'GF-ICI1-plus',
 'GF-ICI2',
 'GF-ICI2-plus'])]

In [ ]:
_ = ir.pl.group_abundance(mdata_colon, groupby="airr:receptor_subtype", target_col="gex:sample_id", )

In [ ]:
_ = ir.pl.group_abundance(mdata_til, groupby="airr:receptor_subtype", target_col="gex:sample_id", )

In [ ]:
mdata.update()

### Receptors in mdata

In [ ]:
_ = ir.pl.group_abundance(mdata, groupby="airr:receptor_subtype", target_col="gex:leiden", )

In [ ]:
_ = ir.pl.group_abundance(mdata, groupby="airr:receptor_subtype", target_col="gex:origin")

In [ ]:
_ = ir.pl.group_abundance(mdata, groupby="airr:chain_pairing", target_col="gex:origin")

In [ ]:
_ = ir.pl.group_abundance(mdata, groupby="airr:chain_pairing", target_col="gex:leiden")

In [ ]:
mu.pl.embedding(
    mdata,
    basis="gex:umap",
    color=["gex:leiden"],
    ncols=3,
    wspace=0.7,
)

### Receptors in mdata colon

In [ ]:
mu.pl.embedding(
    mdata_colon,
    basis="gex:umap",
    color=["gex:leiden"],
    ncols=3,
    wspace=0.7,
)

In [ ]:
_ = ir.pl.group_abundance(mdata_colon, groupby="airr:receptor_subtype", target_col="gex:leiden", )

In [ ]:
_ = ir.pl.group_abundance(mdata_til, groupby="airr:chain_pairing", target_col="gex:leiden")

In [ ]:
_ = ir.pl.group_abundance(mdata, groupby="airr:sample_id", target_col="gex:leiden")

### Receptors in mdata til

In [ ]:
mu.pl.embedding(
    mdata_til,
    basis="gex:umap",
    color=["gex:leiden"],
    ncols=3,
    wspace=0.7,
)

In [ ]:
_ = ir.pl.group_abundance(mdata_til, groupby="airr:receptor_subtype", target_col="gex:leiden", )

In [ ]:
_ = ir.pl.group_abundance(mdata_til, groupby="airr:chain_pairing", target_col="gex:leiden")

## GEX annotation

In [ ]:
# Visualize
sc.pl.umap(adata_gex, color='leiden')

### Cluster 3,4 gex_prot --> cluster 1 in gex_tcr

In [ ]:
sc.pl.umap(adata_gex, color = ["Sell"], cmap="Reds", vmax="p99")

In [ ]:
adata_gex_til = adata_gex[adata_gex.obs["origin"]=="til"]

In [ ]:
adata_gex_colon = adata_gex[adata_gex.obs["origin"]=="colon"]

### GEX annotation til1

In [ ]:
adata_gex_til1 = adata_gex_til[adata_gex_til.obs["leiden"].isin(["1"])]

In [ ]:
adata_gex_til1.var["ribo"] = adata_gex.var_names.str.startswith(("Rps", "Rpl"))

In [ ]:
adata_gex_til1 = adata_gex_til1[:, adata_gex_til1.var['ribo'] == False]

In [ ]:
adata_gex_til1.var["mito"] = adata_gex_til1.var_names.str.startswith(("mt-"))

In [ ]:
adata_gex_til1 = adata_gex_til1[:, adata_gex_til1.var['mito'] == False]

In [ ]:
sc.pp.neighbors(adata_gex_til1, n_neighbors=10)
sc.tl.leiden(adata_gex_til1, resolution=0.3, n_iterations=-1)
sc.tl.umap(adata_gex_til1)

In [ ]:
sc.pl.umap(adata_gex_til1, color='leiden')

In [ ]:
sc.pl.umap(adata_gex_til1, color=['condition',"group"])

In [ ]:
sc.pl.umap(adata_gex_til1, color=["sample_id"])

### Cycling marker

In [ ]:
# Visualize
sc.pl.umap(adata_gex_til1, color='Mki67', cmap="Reds", vmax="p99")

### T differentiation

In [ ]:
# Visualize
sc.pl.umap(adata_gex_til1, color=['Sell','Cd44'], cmap="Reds", vmax="p99")

### T effector into SLEC and MPEC

### Markers from publication SLECs & MPECs

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2034442/

MPECs KLRG1 low, Il7r high, CD122 high, SELF RENEWAL

SLECs KLRG1 high, Il7r low, Cd122 high NOT SELF RENEWAl

Naive CD8 T cells are IL-7Rhi, CD122lo (IL-2/15βR), KLRG1neg and T-betneg and are IL-7 dependent. Early during infection, most effector CD8 T cells become CD122hi and downregulate IL-7R to an intermediate-to-low level, but expression of T-bet and KLRG1 is set depending on their exposure to inflammatory cytokines (e.g. IL-12). Effector CD8 T cells that are exposed to lower levels of inflammation express less T-bet (light blue cells) and begin to upregulate IL-7R to become KLRG1lo IL-7Rhi MPECs (turquoise cells). Effector CD8 T cells that encounter higher levels of inflammatory cytokines express relatively more T-bet and KLRG1 (dark blue cells), stably repress IL-7R and consequentially become KLRG1hi IL-7Rlo SLECs. SLECs become IL-15 dependent, however, IL-15 alone cannot support their long-term persistence or homeostatic turnover and they decline over time. In contrast, MPECs remain dually responsive to IL-7 and IL-15 and preferentially develop into long-lived memory CD8 T cells that can self-renew.

In [ ]:
sc.pl.umap(adata_gex_til1, color = ["Klrg1","Il7r","Il2rb","Itga4","Itgae","Cxcr3","Casp3"], cmap="Reds", vmax="p99")

### Cytotoxic

In [ ]:
sc.pl.umap(adata_gex_til1, color = ["Gzmb","Gzmk","Gzma","Ifng"], cmap="Reds", vmax="p99")

### T act EARLY vs LATE CD25 --> Il2ra

In [ ]:
sc.pl.umap(adata_gex_til1, color = ["Cd69","Il2ra"], cmap="Reds", vmax="p99")

### Top genes from rank genes

In [ ]:
sc.tl.rank_genes_groups(adata_gex_til1, "leiden", method="t-test")
sc.pl.rank_genes_groups(adata_gex_til1, n_genes=25, sharey=False)

In [ ]:
pd.DataFrame(adata_gex_til1.uns["rank_genes_groups"]["names"]).head(20)

In [ ]:
adata_gex_til1.raw = None

In [ ]:
adata_gex_til1.var_names = adata_gex_til1.var["ensembl_id"]

In [ ]:
adata_gex_til1.var_names_make_unique()

In [ ]:
sc.tl.dendrogram(adata_gex_til1, groupby='leiden')

In [ ]:
markers = ['Cdk6', 'Nap1l1', 'Eif1', 'Isg15', 'Icos', 'Dnajb1', 'Irf7']

sc.pl.dotplot(adata_gex_til1, markers, groupby='leiden', dendrogram=True, use_raw=False, gene_symbols="gene_name")

### Exhaustion markers

In [ ]:
sc.pl.umap(adata_gex_til1, color = ["Tox","Lag3"], cmap="Reds", vmax="p99",gene_symbols="gene_name")

In [ ]:
new_cluster_names = ["REC1","REC2","REC3","REC4","REC5",
]
adata_gex_til1.rename_categories("leiden", new_cluster_names)

In [ ]:
sc.pl.umap(adata_gex_til1, color='leiden', legend_loc="on data")

### GEX annotation til2

In [ ]:
adata_gex_til2 = adata_gex_til[adata_gex_til.obs["leiden"].isin(["0","2","3","4","5","6"])]

In [ ]:
adata_gex_til2.var["ribo"] = adata_gex.var_names.str.startswith(("Rps", "Rpl"))

In [ ]:
adata_gex_til2 = adata_gex_til2[:, adata_gex_til2.var['ribo'] == False]

In [ ]:
adata_gex_til2.var["mito"] = adata_gex_til2.var_names.str.startswith(("mt-"))

In [ ]:
adata_gex_til2 = adata_gex_til2[:, adata_gex_til2.var['mito'] == False]

In [ ]:
sc.pp.neighbors(adata_gex_til2, n_neighbors=10)
sc.tl.leiden(adata_gex_til2, resolution=0.3, n_iterations=-1)
sc.tl.umap(adata_gex_til2)

In [ ]:
sc.pl.umap(adata_gex_til2, color='leiden')

In [ ]:
sc.pl.umap(adata_gex_til2, color=['group',"sample_id"])

### Cycling marker

In [ ]:
# Visualize
sc.pl.umap(adata_gex_til2, color='Mki67', cmap="Reds", vmax="p99")

### T differentiation

In [ ]:
# Visualize
sc.pl.umap(adata_gex_til2, color=['Sell','Cd44'], cmap="Reds", vmax="p99")

### T effector into SLEC and MPEC

### Markers from publication SLECs & MPECs

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2034442/

MPECs KLRG1 low, Il7r high, CD122 high, SELF RENEWAL

SLECs KLRG1 high, Il7r low, Cd122 high NOT SELF RENEWAl

Naive CD8 T cells are IL-7Rhi, CD122lo (IL-2/15βR), KLRG1neg and T-betneg and are IL-7 dependent. Early during infection, most effector CD8 T cells become CD122hi and downregulate IL-7R to an intermediate-to-low level, but expression of T-bet and KLRG1 is set depending on their exposure to inflammatory cytokines (e.g. IL-12). Effector CD8 T cells that are exposed to lower levels of inflammation express less T-bet (light blue cells) and begin to upregulate IL-7R to become KLRG1lo IL-7Rhi MPECs (turquoise cells). Effector CD8 T cells that encounter higher levels of inflammatory cytokines express relatively more T-bet and KLRG1 (dark blue cells), stably repress IL-7R and consequentially become KLRG1hi IL-7Rlo SLECs. SLECs become IL-15 dependent, however, IL-15 alone cannot support their long-term persistence or homeostatic turnover and they decline over time. In contrast, MPECs remain dually responsive to IL-7 and IL-15 and preferentially develop into long-lived memory CD8 T cells that can self-renew.

In [ ]:
sc.pl.umap(adata_gex_til2, color = ["Klrg1","Il7r","Il2rb","Itga4","Itgae","Cxcr3","Casp3"], cmap="Reds", vmax="p99")

### Cytotoxic

In [ ]:
sc.pl.umap(adata_gex_til2, color = ["Gzmb","Gzmk","Gzma","Ifng"], cmap="Reds", vmax="p99")

### T act EARLY vs LATE CD25 --> Il2ra

In [ ]:
sc.pl.umap(adata_gex_til2, color = ["Cd69","Il2ra"], cmap="Reds", vmax="p99")

In [ ]:
sc.tl.rank_genes_groups(adata_gex_til2, "leiden", method="t-test")
sc.pl.rank_genes_groups(adata_gex_til2, n_genes=25, sharey=False)

In [ ]:
pd.DataFrame(adata_gex_til2.uns["rank_genes_groups"]["names"]).head(20)

In [ ]:
#adata_gex_til2.raw = None

In [ ]:
adata_gex_til2.var_names = adata_gex_til2.var["ensembl_id"]

In [ ]:
adata_gex_til2.var_names_make_unique()

In [ ]:
sc.tl.dendrogram(adata_gex_til2, groupby='leiden')

In [ ]:
markers = ['S100a10', 'Sell', 'Lef1', 'Ccr7', 'Camk4', 'Foxo1', 'Cmip', 'Fbl']

sc.pl.dotplot(adata_gex_til2, markers, groupby='leiden', dendrogram=True, use_raw=False, gene_symbols="gene_name")

In [ ]:
new_cluster_names = ["EX1","EX2","EX3","EX4","EX5","EX6","EX7","EX8"
]
adata_gex_til2.rename_categories("leiden", new_cluster_names)

In [ ]:
sc.pl.umap(adata_gex_til2, color='leiden', legend_loc="on data")

## COLON

In [ ]:
adata_gex_colon = adata_gex[adata_gex.obs["origin"]=="colon"]

In [ ]:
# Visualize
sc.pl.umap(adata_gex_colon, color=['leiden',"sample_id","group"])

In [ ]:
# Visualize
sc.pl.umap(adata_gex_colon, color=['Cd44','Cd69'], cmap="Reds", vmax="p99")

In [ ]:
adata_gex_colon1 = adata_gex_colon[adata_gex_colon.obs["leiden"].isin(["0","2"])]

In [ ]:
adata_gex_colon1.var["ribo"] = adata_gex_colon1.var_names.str.startswith(("Rps", "Rpl"))

In [ ]:
adata_gex_colon1 = adata_gex_colon1[:, adata_gex_colon1.var['ribo'] == False]

In [ ]:
adata_gex_colon1.var["mito"] = adata_gex_colon1.var_names.str.startswith(("mt-"))

In [ ]:
adata_gex_colon1 = adata_gex_colon1[:, adata_gex_colon1.var['mito'] == False]

In [ ]:
sc.pp.neighbors(adata_gex_colon1, n_neighbors=10)
sc.tl.leiden(adata_gex_colon1, resolution=0.3, n_iterations=-1)
sc.tl.umap(adata_gex_colon1)

In [ ]:
sc.pl.umap(adata_gex_colon1, color='leiden')

### Cycling marker

In [ ]:
# Visualize
sc.pl.umap(adata_gex_colon1, color='Mki67', cmap="Reds", vmax="p99")

### T differentiation

In [ ]:
# Visualize
sc.pl.umap(adata_gex_colon1, color=['Sell','Cd44'], cmap="Reds", vmax="p99")

### T effector into SLEC and MPEC

### Markers from publication SLECs & MPECs

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2034442/

MPECs KLRG1 low, Il7r high, CD122 high, SELF RENEWAL

SLECs KLRG1 high, Il7r low, Cd122 high NOT SELF RENEWAl

Naive CD8 T cells are IL-7Rhi, CD122lo (IL-2/15βR), KLRG1neg and T-betneg and are IL-7 dependent. Early during infection, most effector CD8 T cells become CD122hi and downregulate IL-7R to an intermediate-to-low level, but expression of T-bet and KLRG1 is set depending on their exposure to inflammatory cytokines (e.g. IL-12). Effector CD8 T cells that are exposed to lower levels of inflammation express less T-bet (light blue cells) and begin to upregulate IL-7R to become KLRG1lo IL-7Rhi MPECs (turquoise cells). Effector CD8 T cells that encounter higher levels of inflammatory cytokines express relatively more T-bet and KLRG1 (dark blue cells), stably repress IL-7R and consequentially become KLRG1hi IL-7Rlo SLECs. SLECs become IL-15 dependent, however, IL-15 alone cannot support their long-term persistence or homeostatic turnover and they decline over time. In contrast, MPECs remain dually responsive to IL-7 and IL-15 and preferentially develop into long-lived memory CD8 T cells that can self-renew.

In [ ]:
sc.pl.umap(adata_gex_colon1, color = ["Klrg1","Il7r","Il2rb","Itga4","Itgae","Cxcr3","Casp3"], cmap="Reds", vmax="p99")

### Cytotoxic

In [ ]:
sc.pl.umap(adata_gex_colon1, color = ["Gzmb","Gzmk","Gzma","Ifng"], cmap="Reds", vmax="p99")

### T act EARLY vs LATE CD25 --> Il2ra

In [ ]:
sc.pl.umap(adata_gex_colon1, color = ["Cd69","Il2ra"], cmap="Reds", vmax="p99")

In [ ]:
sc.tl.rank_genes_groups(adata_gex_colon1, "leiden", method="t-test")
sc.pl.rank_genes_groups(adata_gex_colon1, n_genes=25, sharey=False)

In [ ]:
pd.DataFrame(adata_gex_colon1.uns["rank_genes_groups"]["names"]).head(20)

In [ ]:
#adata_gex_til2.raw = None

In [ ]:
adata_gex_colon1.var_names = adata_gex_colon1.var["ensembl_id"]

In [ ]:
adata_gex_colon1.var_names_make_unique()

In [ ]:
sc.tl.dendrogram(adata_gex_colon1, groupby='leiden')

In [ ]:
markers = ['Itgae', 'Eif1', 'Lef1', 'Junb', 'H3f3b']

sc.pl.dotplot(adata_gex_colon1, markers, groupby='leiden', dendrogram=True, use_raw=False, gene_symbols="gene_name")

In [ ]:
adata_gex_colon2 = adata_gex_colon[adata_gex_colon.obs["leiden"].isin(["1","3","4","5"])]

In [ ]:
adata_gex_colon2.var["ribo"] = adata_gex_colon2.var_names.str.startswith(("Rps", "Rpl"))

In [ ]:
adata_gex_colon2 = adata_gex_colon2[:, adata_gex_colon2.var['ribo'] == False]

In [ ]:
adata_gex_colon2.var["mito"] = adata_gex_colon2.var_names.str.startswith(("mt-"))

In [ ]:
adata_gex_colon2 = adata_gex_colon2[:, adata_gex_colon2.var['mito'] == False]

In [ ]:
sc.pp.neighbors(adata_gex_colon2, n_neighbors=10)
sc.tl.leiden(adata_gex_colon2, resolution=0.3, n_iterations=-1)
sc.tl.umap(adata_gex_colon2)

In [ ]:
sc.pl.umap(adata_gex_colon2, color='leiden')

### Cycling marker

In [ ]:
# Visualize
sc.pl.umap(adata_gex_colon2, color='Mki67', cmap="Reds", vmax="p99")

### T differentiation

In [ ]:
# Visualize
sc.pl.umap(adata_gex_colon2, color=['Sell','Cd44'], cmap="Reds", vmax="p99")

### T effector into SLEC and MPEC

### Markers from publication SLECs & MPECs

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2034442/

MPECs KLRG1 low, Il7r high, CD122 high, SELF RENEWAL

SLECs KLRG1 high, Il7r low, Cd122 high NOT SELF RENEWAl

Naive CD8 T cells are IL-7Rhi, CD122lo (IL-2/15βR), KLRG1neg and T-betneg and are IL-7 dependent. Early during infection, most effector CD8 T cells become CD122hi and downregulate IL-7R to an intermediate-to-low level, but expression of T-bet and KLRG1 is set depending on their exposure to inflammatory cytokines (e.g. IL-12). Effector CD8 T cells that are exposed to lower levels of inflammation express less T-bet (light blue cells) and begin to upregulate IL-7R to become KLRG1lo IL-7Rhi MPECs (turquoise cells). Effector CD8 T cells that encounter higher levels of inflammatory cytokines express relatively more T-bet and KLRG1 (dark blue cells), stably repress IL-7R and consequentially become KLRG1hi IL-7Rlo SLECs. SLECs become IL-15 dependent, however, IL-15 alone cannot support their long-term persistence or homeostatic turnover and they decline over time. In contrast, MPECs remain dually responsive to IL-7 and IL-15 and preferentially develop into long-lived memory CD8 T cells that can self-renew.

In [ ]:
sc.pl.umap(adata_gex_colon2, color = ["Klrg1","Il7r","Il2rb","Itga4","Itgae","Cxcr3","Casp3"], cmap="Reds", vmax="p99")

### Cytotoxic

In [ ]:
sc.pl.umap(adata_gex_colon2, color = ["Gzmb","Gzmk","Gzma","Ifng"], cmap="Reds", vmax="p99")

### T act EARLY vs LATE CD25 --> Il2ra

In [ ]:
sc.pl.umap(adata_gex_colon2, color = ["Cd69","Il2ra"], cmap="Reds", vmax="p99")

In [ ]:
sc.tl.rank_genes_groups(adata_gex_colon2, "leiden", method="t-test")
sc.pl.rank_genes_groups(adata_gex_colon2, n_genes=25, sharey=False)

In [ ]:
pd.DataFrame(adata_gex_colon2.uns["rank_genes_groups"]["names"]).head(20)

In [ ]:
#adata_gex_til2.raw = None

In [ ]:
adata_gex_colon2.var_names = adata_gex_colon2.var["ensembl_id"]

In [ ]:
adata_gex_colon2.var_names_make_unique()

In [ ]:
sc.tl.dendrogram(adata_gex_colon2, groupby='leiden')

In [ ]:
markers = ['Igfbp4', 'Gm42418', 'Cox6c', 'Pfn1', 'Cd52']

sc.pl.dotplot(adata_gex_colon2, markers, groupby='leiden', dendrogram=True, use_raw=False, gene_symbols="gene_name")

## naive markers in colon 
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4810120/#:~:text=CD8%20Cell%20Subsets%20and%20Cell%20Differentiation&text=Naive%20T%20cells%20differentiate%20into,are%20shown%20in%20Figure%204.